In [10]:
import os
import tensorflow as tf
import IPython.display as display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import PIL
import glob

mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

In [3]:
#WARNING! This is slow to run! And you only reed to run it once.
import kagglehub
import tensorflow_hub as hub

path = kagglehub.model_download("google/arbitrary-image-stylization-v1/tensorFlow1/256")
hub_model = hub.load(path)

/Users/megan/Documents/art/jan_collab/style_transfer/.venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
/Users/megan/Documents/art/jan_collab/style_transfer/.venv/lib/python3.12/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [4]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

In [5]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [6]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3, expand_animations=False)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:2], tf.float32)
  long_dim = tf.reduce_max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

In [7]:
def load_img_fixed(path_to_img, size=256):
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3, expand_animations=False)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = tf.image.resize(img, [size, size])
  img = img[tf.newaxis, :]
  return img


def get_style_embeddings(style_paths):
    embeddings = []
    for path in style_paths:
        try:
            img = load_img_fixed(path) # Ensure this returns a float32 tensor [1, H, W, 3]
            embeddings.append(img)
        except Exception as e:
            print(f"Error loading style image {path}: {e}")
            continue
    return embeddings

In [8]:
def imagepaths_list_from_folder(folder_path):
    return [f"{folder_path}/{f}" for f in os.listdir(folder_path) if not f.startswith('.')]

In [11]:
content_paths = imagepaths_list_from_folder("../data/content")
style_paths = imagepaths_list_from_folder("../data/style")

In [ ]:
# APPLY EVERY STYLE IMAGE INDIVIDUALLY
num = 0

for style_path in style_paths:
    style_img = load_img(style_path)
    content_img = load_img(content_paths[0])
    stylized_image = hub_model(tf.constant(content_img), tf.constant(style_img))[0]
    output_image = tensor_to_image(stylized_image)
    output_image.save(f"../outputs/all_styles_grid2/{num}.png")
    num += 1

In [ ]:
# RANDOM STYLE ANIMATION
animation_frames = sorted(glob.glob('../data/animation/*.png'))
style_paths = imagepaths_list_from_folder("../data/style")

num = 0

for frame in animation_frames:
    random_style_int = np.random.randint(0, len(style_paths))
    try:
        style_img = load_img(style_paths[random_style_int])
    except Exception as e:
        print(f"Error loading style image {style_path}: {e}")
        continue
    content_img = load_img(frame)
    stylized_image = hub_model(tf.constant(content_img), tf.constant(style_img))[0]
    output_image = tensor_to_image(stylized_image)
    output_image.save(f"../outputs/cube_animation_1/{num}.png")
    num += 1

KeyboardInterrupt: 

In [ ]:
# AVERAGE STYLE ANIMATION
animation_frames = sorted(glob.glob('../data/animation/*.png'))
style_paths = imagepaths_list_from_folder("../data/style2")

style_embeddings = get_style_embeddings(style_paths)
avg_style = tf.reduce_mean(tf.concat(style_embeddings, axis=0), axis=0, keepdims=True)

num = 0

for frame in animation_frames:
    random_style_int = np.random.randint(0, len(style_paths))
    content_img = load_img(frame)
    stylized_image = hub_model(tf.constant(content_img), tf.constant(avg_style))[0]
    output_image = tensor_to_image(stylized_image)
    output_image.save(f"../outputs/cube_animation_2/{num}.png")
    num += 1

KeyboardInterrupt: 

In [ ]:
# TRANSITIONING STYLES ANIMATION
import numpy as np
import tensorflow as tf

style_paths = imagepaths_list_from_folder("../data/style2")
style_tensors = [load_img_fixed(p) for p in style_paths]
num_styles = len(style_tensors)

animation_frames = sorted(glob.glob('../data/animation/*.png'))
num_frames = len(animation_frames)

for i, frame_path in enumerate(animation_frames):
    # Determine which two styles we are between
    # This maps the current frame index to a float position in the style list
    style_idx_float = (i / (num_frames - 1)) * (num_styles - 1)
    idx1 = int(np.floor(style_idx_float))
    idx2 = int(np.ceil(style_idx_float))
    
    # Calculate the blend weight (0.0 to 1.0)
    fraction = style_idx_float - idx1
    
    # Interpolate between the two style images
    # This creates a 'morphed' style image for this specific frame
    interpolated_style = (1 - fraction) * style_tensors[idx1] + fraction * style_tensors[idx2]
    
    # Run Stylization
    content_img = load_img(frame_path)
    stylized_image = hub_model(tf.constant(content_img), tf.constant(interpolated_style))[0]
    
    output_image = tensor_to_image(stylized_image)
    output_image.save(f"../outputs/cube_animation_3/{i}.png")

KeyboardInterrupt: 

In [12]:
#SINGLE STYLE
content_path = "../data/content/grid1.jpg"
style_path = "../data/style/pomelo.jpeg"

content_img = load_img(content_path)
style_img = load_img(style_path)

# Find the next available number
import re
existing = [f for f in os.listdir("../outputs") if re.match(r'^\d+\.png$', f)]
if existing:
    nums = [int(re.match(r'^(\d+)\.png$', f).group(1)) for f in existing]
    num = max(nums) + 1
else:
    num = 1

stylized_image = hub_model(tf.constant(content_img), tf.constant(style_img))[0]

# Save the stylized image
output_image = tensor_to_image(stylized_image)
output_image.save(f"../outputs/{num}.png")
print(f"Saved to outputs/{num}.png")

Saved to outputs/6.png
